Alex Beckwith - HW8 

Tips

1###    Change ds = -A\B; to ds = -0.1*(A\B);

2### s0 > 1

3### lagrange mults0 all = 1

4### In my attempt to create a nice nonlinear problem for the second one, I did in fact give a constraint that is not globally convex. So if you are comparing your answer with MATLAB fmincon or another solver, you may not get the same answer and you should be able to get different answers with different initializations. 


In [141]:
#imports and qual of life funcs
from numpy import array,matmul,log,sqrt,zeros,diag
from numpy.linalg import norm,lstsq
from numpy.random import randint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from interior_point_helpers import create_test_array,multi_decorator

###set8 problem1###

min f(x) = x1^2 + x2^2 + x3^2 + x4^2 + x5^2 

st x1 - x2 + x3 -3 = 0

st 3 - (x3 + x4) <= 0

st x5 + 2 <= 0 

###set8 problem2###

min x1^4+x2^4+x3^4

st sin(x1+x2+x3) - sqrt(2)/2 = 0

st -(x1^2+x2^2-1) <= 0

In [142]:
#constants
dx = 1e-4
x_max = 10

def f1(x:array):
    sum_f=sum([xn**2 for xn in x])
    return sum_f

def g1(x:array):
    g1 = 3 - x[2] - x[3]
    g2 = x[4] + 2
    return [g1,g2]

def h1(x:array):
    return x[0] - x[1] + x[2] - 3

def f2(x:array,idx_range=3):
    return sum([x[i]**4 for i in range(idx_range)])

def g2(x:array,idx_range=3):
    x_sum = sum(x[:idx_range])
    sub = np.sqrt(2)/2
    return x_sum-sub

def h2(x:array,idx_range=2):
    xs = [xn**2 for xn in x]
    return -(sum(xs)-1)

def prob_setup(pn:int,x_max=10):
    x = lambda y: array([randint(x_max) for x in range(y)])
    func_dict = {
        "key":["f_min","g_ineq","h_eq","x_len"],
        1:[f1,g1,h1,5],
        2:[f2,g2,h2,3]
    }
    p_list = func_dict[pn]
    f = p_list[0]
    g = p_list[1]
    h = p_list[2]
    x_len = p_list[3]
    test_x = x(x_len)
    try:
        [func(test_x) for func in p_list[:3]]
    except:
        print(func_dict["key"]) 
    return f,g,h

In [151]:
@multi_decorator
def grad(func,x:array,dx=dx):
    a = len(x)
    grad_array = zeros(a)

    for j in range(a):
        dxi = zeros(a)
        dxi[j] = dx
        grad_array[j] = (func(x+dxi)-func(x))/dx
    return grad_array

@multi_decorator
def hess(func,x:array,dx=dx):
    try:
        a = len(x)
    except:
        a = 1
    H = zeros(a**2).reshape(a,a)

    for j in range(a):
        dxj = zeros(a)
        dxj[j]=dx
        def dfj(func=func,x=x,dxi=dxj):
            return (func(x+dxi)-func(x))/dx
        
        for k in range(j,a):
            dxk = zeros(a)
            dxk[k]=dx
            dfjk = (dfj(x=x+dxk)-dfj())/dx
            H[j][k] = H[k][j] = dfjk
    return H

In [152]:
@multi_decorator
def jac(func,x,dx=dx):
    lx = len(x)
    try:
        l = len(func(x))
    except:
        l = 1
    J = zeros(l*lx).reshape(lx,l)
    for j in range(lx):
        dxj = zeros(lx)
        dxj[j] = dx
        J[:][j] = (func(x+dxj)-func(x))/dx
    return J.T

In [153]:
def mega_hess(x,f,g,h,λ,y):   
    H = hess(f,x)
    Hg = hess(g,x)
    Hh = hess(h,x)
    scalars = [λ,y]
    for i,gh in enumerate([Hg,Hh]):
        mult = scalars[i]
        try:
            a = len(mult)
            first = mult[0]*gh[0]
            for j in range(1,a):
                first+=mult[j]*gh[j]        
        except:
            first = mult*gh        
        H += first
    return H

In [154]:
#need slack variable
xl = 5
x = array([np.random.randint(-100,100) for y in range(xl)])
def slack(g,x):
    gx = array(g(x))
    gx = gx.reshape(len(gx),1)
    nx = zeros(len(gx)).reshape(len(gx),1)
    slacks = lstsq(gx,nx,rcond=None)
    return slacks[0]

#need merit function
def merit():
    pass

In [155]:
x=create_test_array(length=5)
def barrier_func(mu,x,func=g):
    funcx = func(x)
    logx = sum([log(xn) for xn in x])
    ret = funcx-(mu*logx)
    return ret

mus = [mu for mu in range(1,10)]
vals = [barrier_func(mu,x) for mu in mus]

In [226]:
class Step():
    def H(self):
        full_H = mega_hess( self.x,
                            self.f,
                            self.g,
                            self.h,
                            self.λ,
                            self.y)
        return full_H

    def A(self):
        a1 = self.Hn
        a2 = zeros(self.nx*self.ng).reshape(self.nx,self.ng)
        a3 = self.Jh.T
        a4 = self.Jg.T
        return [a1,a2,a3,a4]
    
    def B(self):
        b1 = zeros(self.nx*self.ng).reshape(self.ng,self.nx)
        try:
            b2 = array([[self.s],[self.λ]])
        except:
            if (len(self.λ)>1 and self.len_s>1):
                b2 = matmul(diag(self.s),diag(self.λ))
            elif (len(self.λ)>1 or self.len_s>1):
                b2 = diag(self.s)*self.λ
            else:
                b2 = diag(array([self.s,self.λ]))
        b3 = zeros(self.nh*self.ng).reshape(self.ng,self.nh)
        try:
            b4 = diag(self.s)
        except:
            b4 = array([[self.s]])
        #print([b1,b2,b3,b4])
        return [b1,b2,b3,b4]

    def C(self):
        c1 = self.Jh
        c2 = zeros(self.nh*self.ng).reshape(self.nh,self.ng)
        c3 = zeros(self.nh**2).reshape(self.nh,self.nh)
        c4 = zeros(self.nh*self.ng).reshape(self.nh,self.ng)
        return [c1,c2,c3,c4]

    def D(self):
        d1 = self.Jg
        d2 = diag([1 for i in range(self.ng)])
        d3 = zeros(self.nh*self.ng).reshape(self.ng,self.nh)
        d4 = zeros(self.ng**2).reshape(self.ng,self.ng)
        return [d1,d2,d3,d4]

    def concat_mat_row(self,mat_row:array):
        for i,a in enumerate(mat_row):

            a_shape_len = len(a.shape)

            if a_shape_len>2:
                dims = [a.shape[i] for i in range(a_shape_len)]
                flat_dim = 1
                for dim in dims:
                    flat_dim*=dim
                a = a.reshape(1,int(flat_dim/a.shape[0]))
            elif a_shape_len<2:
                a = a.reshape(1,a_shape_len)
            print(a.shape)
            if (a.shape[0]!=1 and a.shape[0]!=a.shape[-1]):
                a0 = a.shape[0]
                a1 = a.shape[-1]
                a = a.reshape(a1,a0)
            
            mat_row[i] = a
        try:
            ret = np.concatenate(mat_row,axis=1)
            return ret
        except:
            print(mat_row)

    def corr_shape(row):
        pass
        

    def stack_rows(self):
        to_stack = []
        A = self.A()
        B = self.B()
        C = self.C()
        D = self.D()
        for row in [A,B,C,D]:
            print([x.shape for x in row])
            cat = self.concat_mat_row(row)
            to_stack.append(cat)

        print([z.shape for z in to_stack])
        return np.concatenate(to_stack,axis=0)

    def direct_step(self):
        a1 = grad(self.f,self.x)
        a2 = (self.Jh.T*self.y).sum(axis=1)
        a3 = np.matmul(self.Jg.T,self.λ)
        an = [a.ravel() for a in [a1,a2,a3]]
        a = np.zeros(len(a1))
        for i in range(len(a1)):
            for a_part in an:
                a[i]+=a_part[i]
        try:
            b = np.matmul(np.diag(self.s),self.λ)-self.mu*self.e
        except:
            b = (self.s*self.λ)-self.mu*self.e
        c = array(self.h(self.x))
        d = array(self.g(self.x))+self.s
        return_array = []
        for row in [a,b,c,d]:
            if len(row.shape)>0:
                for val in row:
                    return_array.append(val)
            else:
                return_array.append(row)
        return array([-r for r in return_array])

    def f_mu(self):
        fx = self.f(self.x)
        mu_s = self.mu*sum([np.log(si) for si in self.s])
        #print(fx,mu_s)
        return fx-mu_s

    def __init__(self,x,s,λ,y,mu,pn):
        self.f,self.g,self.h = prob_setup(pn)
        self.x = x
        self.s = s
        #self.f = self.f_mu
        try:
            self.len_s = len(self.s)
        except:
            self.len_s = 1
        self.e = np.ones(self.len_s)
        self.λ = array(λ)
        self.y = array(y)
        self.mu = mu
        self.Hn = self.H()
        self.Jg = jac(self.g,self.x)
        self.Jh = jac(self.h,self.x)
        self.nx = len(self.x)
        self.ng = self.len_s
        try:
            self.nh = len(self.h(self.x))
        except:
            self.nh = 1
        self.stack = self.stack_rows()


In [231]:
"""
s = slack variables
mu = weight of barrier function
"""

class FullSystem():
    def make_new_vals(self):
        step = self.cur_step
        orig_vals = [step.x,step.s,step.y,step.λ]
        idx_counter = 0
        lens = [len(x) for x in orig_vals]
        deltas = 0.1*self.deltas
        new_vals = []
        for i,val in enumerate(orig_vals):
            to_add = deltas[idx_counter:idx_counter+lens[i]]
            new_vals.append(val+to_add)
            idx_counter+=lens[i]
        return new_vals

    def calc_deltas(self,step):
        big_matrix = step.stack
        print(big_matrix,len(big_matrix.shape),big_matrix.shape)
        
        dir_step = step.direct_step()
        dir_step = dir_step.reshape(len(dir_step),1)
        print(dir_step,len(dir_step.shape),dir_step.shape)
        try:
            deltas = lstsq(big_matrix,dir_step,rcond=None)
        except:
            #dir_step=dir_step.T
            dir_step=dir_step.reshape(len(dir_step),1)
            print(dir_step,dir_step.shape)
            deltas = lstsq(big_matrix,dir_step,rcond=None)
        return deltas[0]

    def rec_entry(self,iter:int):
            rec_types = ["iter","x","s","λ","y","mu"]
            new_rec = {"iter":iter,"x":self.x,"s":self.s,"λ":self.λ,"y":self.y,"mu":self.mu}
            for rec in rec_types:
                record_list = self.return_dict[rec]
                record_list.append(new_rec[rec])
                self.return_dict[rec] = record_list

    def main(self):
        mus = np.linspace(self.min_mu,self.max_mu,self.max_iter+1)
        steps = range(2,self.max_iter+1)
        for i in steps:
            norm1 = norm(self.x)
            new_vals = self.make_new_vals()
            self.x = new_vals[0]
            self.s = new_vals[1]
            self.y = new_vals[2]
            self.λ = new_vals[3]
            self.cur_step = Step(self.x,self.s,self.λ,self.y,mus[i],self.pn)
            norm2 = norm(self.x)
            dif = norm2-norm1
            if i%self.rec_factor==0:
                self.rec_entry(i)
            if i%(int(self.max_iter/10))==0:
                print(i)
            if abs(dif)<self.min_dif:
                print("converged")
                converge_iter = i
                break
        return pd.DataFrame.from_dict(self.return_dict)
 


    def __init__(self,pn):
        self.pn = pn
        f,g,h = prob_setup(pn)
        self.rec_factor = 250
        self.min_dif = 1e-6
        self.max_iter = self.converge_iter = int(1e4)
        self.mu = self.min_mu = 1e-4
        self.max_mu = 1e-4
        self.x = np.array([2,1,2,2,-3])
        self.s = -array(g(self.x))  #the lstsq method only returned 1d zeros                            
        try:                        #to update s so I used this
            self.λ = np.ones(len(g(x)))
        except:
            self.λ = np.ones(1)
        try:
            self.y = np.ones(len(h(x)))
        except:
            self.y = np.ones(1)
        self.cur_step = Step(self.x,self.s,self.λ,self.y,self.mu,self.pn)
        self.deltas = self.calc_deltas(self.cur_step)
        self.return_dict = {
            "iter":[1],
            "x":[self.cur_step.x],
            "s":[self.cur_step.s],
            "λ":[self.cur_step.λ],
            "y":[self.cur_step.y],
            "mu":[self.cur_step.mu]
            }
        
        
system = FullSystem(1)
system.main()


[(5, 5), (5, 2), (5, 1), (5, 1, 2)]
(5, 5)
(5, 2)
(5, 1)


ValueError: cannot reshape array of size 10 into shape (1,2)

In [ ]:
def definitions(type="dict"):
    def_dict = {
    "Jg":"the Jacobian of the constraint function g",
    "Jh":"the Jacobian of the constraint function h",
    "S":"diag(s)",
    "lamb":"the Lagrange multiplier vector associated with constraints g",
    "Lamb":"diag(lamb)",
    "y":"the Lagrange multiplier vector associated with h",
    "e":"the vector of ones the same size as g"
    }
    def_mat = array([
                    ["H",0,"JhT","JgT"],
                    [0,"Lamb*e^s_hat",0,"e^s_hat"],
                    ["Jh",0,0,0],
                    ["Jg","e^s_hat",0,0]
                    ])
    if type=="dict":
        for d in def_dict.keys():
            print(d+": ",def_dict[d])
    else:
        print(def_mat)
definitions("dict")

Jg:  the Jacobian of the constraint function g
Jh:  the Jacobian of the constraint function h
S:  diag(s)
lamb:  the Lagrange multiplier vector associated with constraints g
Lamb:  diag(lamb)
y:  the Lagrange multiplier vector associated with h
e:  the vector of ones the same size as g
